In [6]:
import matplotlib.image as img
import matplotlib.pyplot as plt
from scipy.cluster.vq import whiten
from scipy.cluster.vq import kmeans
import pandas as pd

In [7]:
# function to pull top 3 colors of each album art
def get_top_colors(art):
    r = []
    g = []
    b = []
    
    for row in art:
        for temp_r, temp_g, temp_b in row:
            r.append(temp_r)
            g.append(temp_g)
            b.append(temp_b)
            
    df = pd.DataFrame({'red'   : r,
                       'green' : g,
                       'blue'  : b})
    
    df['scaled_color_red'] = whiten(df['red'])
    df['scaled_color_blue'] = whiten(df['blue'])
    df['scaled_color_green'] = whiten(df['green'])

    cluster_centers, _ = kmeans(df[['scaled_color_red',
                                    'scaled_color_blue',
                                    'scaled_color_green']], 3)

    dominant_colors = []

    red_std, green_std, blue_std = df[['red',
                                       'green',
                                       'blue']].std()

    for cluster_center in cluster_centers:
        red_scaled, green_scaled, blue_scaled = cluster_center
        dominant_colors.append((
            red_scaled * red_std / 255,
            green_scaled * green_std / 255,
            blue_scaled * blue_std / 255
        ))
        
    return dominant_colors

#  citation - https://www.geeksforgeeks.org/extract-dominant-colors-of-an-image-using-python/

In [11]:
# function to classify color based on RBG value's manhattan distance to the below colors
def classify(rgb_tuple):
    colors = {'black'  :(0,0,0),
              'white'  :(1,1,1),
              'red'    :(1,0,0),
              'lime'   :(0,1,0),
              'blue'   :(0,0,1),
              'yellow' :(1,1,0),
              'cyan'   :(0,1,1),
              'magenta':(1,0,1),
             }
    
    manhattan = lambda x,y : abs(x[0] - y[0]) + abs(x[1] - y[1]) + abs(x[2] - y[2]) 
    distances = {k: manhattan(v, rgb_tuple) for k, v in colors.items()}
    color = min(distances, key=distances.get)
    return color

# citation - https://stackoverflow.com/questions/36439384/classifying-rgb-values-in-python

In [13]:
# creating function add column distinguishing black and white images from regular ones
def b_w(test):
    count = 0
    for i in test:
        if i == 'black' or i == 'white':
            count += 1
    if count == 3:
        return 'black and white image'
    else:
        return 'colored image'